In [34]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import os

import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.python.ops.numpy_ops import np_config

import json

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


2022-06-25 00:03:19.127124: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-25 00:03:19.179466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 00:03:19.205398: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 00:03:19.206059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [25]:
tf.config.run_functions_eagerly(True)
# tf.debugging.set_log_device_placement(False)

In [5]:
tf.executing_eagerly()

True

In [6]:
ds,info = tfds.load('quickdraw_bitmap',split="train", with_info=True)
print(info)

tfds.core.DatasetInfo(
    name='quickdraw_bitmap',
    full_name='quickdraw_bitmap/3.0.0',
    description="""
    The Quick Draw Dataset is a collection of 50 million drawings across 345 categories, contributed by players of the game Quick, Draw!. The bitmap dataset contains these drawings converted from vector format into 28x28 grayscale images
    """,
    homepage='https://github.com/googlecreativelab/quickdraw-dataset',
    data_path='/home/elice/tensorflow_datasets/quickdraw_bitmap/3.0.0',
    download_size=36.82 GiB,
    dataset_size=19.71 GiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=345),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=50426266, num_shards=256>,
    },
    citation="""@article{DBLP:journals/corr/HaE17,
      author    = {David Ha and
                   Douglas Eck},
 

2022-06-25 00:03:30.569852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 00:03:30.570571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 00:03:30.571129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 00:03:30.572109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-25 00:03:30.572773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [7]:
train_ds = tfds.load('quickdraw_bitmap',split="train[:60%]")
test_ds = tfds.load('quickdraw_bitmap',split="train[60%:80%]")
val_ds = tfds.load('quickdraw_bitmap',split="train[80%:]")

In [53]:
def normalize(dataset):    
    image, label = tf.cast(dataset["image"], tf.float32) / 255.0, dataset["label"]
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, (32,32))
    return image, label

In [56]:
# 용량 관련 오류가 발생하면 이 곳의 배치 사이즈를 줄이면 됩니다.
train_dataset = train_ds.map(normalize).batch(1536)
test_dataset = test_ds.map(normalize).batch(1536)
val_dataset = val_ds.map(normalize).batch(1536)

AttributeError: in user code:

    File "/tmp/ipykernel_14619/2045070421.py", line 3, in normalize  *
        image = gray2rgb(image)
    File "/tmp/ipykernel_14619/274547884.py", line 2, in gray2rgb  *
        image = cv2.cvtColor((i[:,:,0].astype(np.float32)), cv2.COLOR_GRAY2RGB)

    AttributeError: Tensor object has no attribute 'astype'. 
            If you are looking for numpy-related methods, please run the following:
            from tensorflow.python.ops.numpy_ops import np_config
            np_config.enable_numpy_behavior()
          


In [17]:
input_shape = (32,32,3)
dropout_rate = 0.5
epochs = 10

In [11]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [12]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 verbose=1)

In [13]:
efficientnetb4 = EfficientNetB4(weights="imagenet",include_top=False, input_shape=input_shape, drop_connect_rate=dropout_rate)
efficientnetb4.trainable=True # efficientnetb4의 학습을 허용.

In [14]:
model4 = Sequential() #새 Sequential 모델을 만듬 
model4.add(Input(shape=input_shape)) #인풋을 이미지 사이즈로 설정
model4.add(efficientnetb4) # efficientnetb0 추가
model4.add(layers.GlobalAveragePooling2D()) # 풀링 레이어를 추가
model4.add(layers.Dropout(dropout_rate)) # 드롭아웃 레이어를 추가
model4.add(Dense(345, activation="softmax")) #마지막 덴스 레이어를 추가. 예측할 클래스의 개수만큼이 아웃풋이 된다. 
model4.summary() #모델 확인

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb4 (Functional)  (None, 1, 1, 1792)       17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 345)               618585    
                                                                 
Total params: 18,292,408
Trainable params: 18,167,201
Non-trainable params: 125,207
_________________________________________________________________


In [15]:
model4.compile(loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=["accuracy"]) 
#loss는 sparse_categorical_crossentropy, optimizer는 Adam을 사용. 각 에폭당 정확도를 통해 모델의 성능을 모니터링함

In [18]:
history = model4.fit(train_dataset,#모델을 학습합니다. 
                    epochs = epochs,
                    batch_size = 1024,
                    callbacks=[cp_callback])

Epoch 1/18


ValueError: Input 0 of layer "sequential" is incompatible with the layer: expected shape=(None, 32, 32, 3), found shape=(2048, 28, 28, 1)

In [ ]:
model4.save("image_model.h5")